# Data Preprocessing

- akan melakukan preprocessing corpus hasil scraping website halodoc menggunakan stopwords indonesia dan stemming dengan sastrawi
- untuk fine-tuning model indoBERT

## Import and Initializing

In [20]:
# import
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# load stopwords
stop_words = set(stopwords.words('indonesian'))

# load stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()



In [21]:
# load dataset
df = pd.read_csv('scraping/halodoc1-1000.csv')
print(df.shape)

# check dataset
df.head()

(9986, 5)


,url,title,title_length,content,content_count
0,https://www.halodoc.com/ketahui-segala-hal-men...,Ketahui Segala Hal Mengenai COVID-19,38,“Infeksi COVID-19 adalah salah satu penyakit y...,6532
1,https://www.halodoc.com/bagaimana-cara-merawat...,Bagaimana Cara Merawat Karies Gigi?,37,“Ada banyak pilihan perawatan medis dan rumaha...,5993
2,https://www.halodoc.com/cegah-burnout-ini-tand...,"Cegah Burnout, Ini Tanda Pekerja Butuh Piknik...",61,“Burnout merupakan kondisi ketika seseorang m...,4982
3,https://www.halodoc.com/kenalan-dengan-terapi-...,Kenalan dengan Terapi ABA untuk Anak Autis,44,“Salah satu cara untuk mengurangi gejala auti...,4855
4,https://www.halodoc.com/10-herbal-untuk-mereda...,10 Herbal untuk Meredakan Sakit Perut Bagian ...,52,“Ada banyak herbal yang dapat membantu mereda...,3638


In [22]:
# check null values
df.isnull().sum()

url              0
title            0
title_length     0
content          0
content_count    0
dtype: int64

In [ ]:
# reassign 'title_length' and 'content_length' in df dengan panjang dari 'title' dan 'content' yang baru (word count)
df['title_length'] = df['title'].apply(lambda x: len(x.split()))
df['content_count'] = df['content'].apply(lambda x: len(x.split()))

df.head()

## Data Preprocessing

In [23]:
# check duplicates
print(df.duplicated().sum())
print(f"Ukuran dataset sebelum preprocessing: {df.shape}")

# drop duplicates
df.drop_duplicates(inplace=True)
print(f"Ukuran dataset setelah preprocessing: {df.shape}")

5
Ukuran dataset sebelum preprocessing: (9986, 5)
Ukuran dataset setelah preprocessing: (9981, 5)


### Remove unnecessary string in 'content'

In [24]:
# tbd
# tbd
# tbd
# tbd
# tbd
# tbd
# tbd
# tbd
# tbd
# tbd
# tbd
# tbd
# tbd
# tbd

### Stopwords Removal, Stemming, etc. (probably wont be used)

In [25]:
# # remove numbers that are not attached to words
# def remove_numbers(text):
#     text = re.sub(r'(?<!\S)\d+(?!\S)', '', text)
#     return text

# df['title'] = df['title'].apply(remove_numbers)
# df['content'] = df['content'].apply(remove_numbers)
# print('hapus angka beridiri sendiri selesai')

# df.head()

In [26]:
# # remove punctuation
# def remove_punctuation(text):
#     text = re.sub(r'[^\w\s]', '', text)
#     return text

# df['title'] = df['title'].apply(remove_punctuation)
# df['content'] = df['content'].apply(remove_punctuation)
# print('hapus tanda baca selesai')

# df.head()

# # remove numbers
# def remove_numbers(text):
#     text = re.sub(r'\d+', '', text)
#     return text

# df['title'] = df['title'].apply(remove_numbers)
# df['content'] = df['content'].apply(remove_numbers)
# print('hapus angka selesai')

# df.head()

# # stopword removal + lowercasing
# def remove_stopwords(text):
#     text = [word.lower() for word in text.split() if word.lower() not in stop_words]
#     return " ".join(text)

# df['title'] = df['title'].apply(remove_stopwords)
# df['content'] = df['content'].apply(remove_stopwords)
# print('stopword removal dan lowercase selesai')

# df.head()

# # stemming
# def stemming(text):
#     text = [stemmer.stem(word) for word in text.split()]
#     return " ".join(text)

# df['title'] = df['title'].apply(stemming)
# df['content'] = df['content'].apply(stemming)
# print('stemming selesai')

# df.head()

In [28]:
# # summarization 'content' with indoBERT (not yet)
# # summarization 'content' with BERT

# # import library
# import torch
# from transformers import BertTokenizer, BertModel, BertForMaskedLM
# from transformers import pipeline

# # load model
# tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
# model = BertModel.from_pretrained('indobenchmark/indobert-base-p1')

# # summarize 'content'
# summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
# df['summary'] = df['content'].apply(lambda x: summarizer(x, max_length=100, min_length=30, do_sample=False)[0]['summary_text'])

# df.head()


In [29]:
# save to csv
df.to_csv('halodoc1-1000_preprocessed.csv', index=False)

## Tokenization

In [30]:
# tokenization df to list
title = df['title'].tolist()
content = df['content'].tolist()


(title[:5], content[:5])

([' Ketahui Segala Hal Mengenai COVID-19 ',
  ' Bagaimana Cara Merawat Karies Gigi? ',
  ' Cegah Burnout, Ini Tanda Pekerja Butuh Piknik dan Istirahat ',
  ' Kenalan dengan Terapi ABA untuk Anak Autis ',
  ' 10 Herbal untuk Meredakan Sakit Perut Bagian Bawah '],
 ['“Infeksi COVID-19 adalah salah satu penyakit yang perlu diwaspadai. Gejalanya bisa ringan seperti flu biasa, hingga berat dan sebabkan kematian.”Halodoc, Jakarta - COVID-19 adalah infeksi yang disebabkan oleh virus corona atau SARS-CoV-2. Penyakit ini sempat menyebabkan jutaan jiwa dari seluruh dunia menjadi korban.Infeksi COVID-19 memiliki gejala umum berupa gangguan sistem pernapasan ringan hingga berat. Pada orang dengan komorbid atau yang mengidap penyakit kronis sebelumnya, gejala bisa jadi lebih parah dan bahkan menyebabkan kematian.Hal-Hal Seputar COVID-19Untuk lebih memahami seputar infeksi COVID-19, ada beberapa hal yang perlu kamu ketahui. Berikut pembahasannya:COVID-19 Sama dengan Virus SARS?COVID-19 adalah penyak